In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.compose import ColumnTransformer

In [2]:
# Đọc dữ liệu
train = pd.read_csv('df_saturday_train.csv')
test = pd.read_csv('df_saturday_test.csv')

# for col in train.columns:
#     train[col] = train[col].fillna(train[col].mode()[0])

# for col in test.columns:
#     test[col] = test[col].fillna(test[col].mode()[0])

In [ ]:
class KalmanFilter:
    def __init__(self, A=1, H=1, Q=1e-5, R=10, initial_state=0, initial_uncertainty=1):
        self.A = A
        self.H = H
        self.Q = Q
        self.R = R
        self.x = initial_state
        self.P = initial_uncertainty

    def predict(self):
        self.x = self.A * self.x
        self.P = self.A * self.P * self.A + self.Q
        return self.x

    def update(self, z):
        K = self.P * self.H / (self.H * self.P * self.H + self.R)
        self.x = self.x + K * (z - self.H * self.x)
        self.P = (1 - K * self.H) * self.P
        return self.x
    
ts_train = train['Listening_Time_minutes']

# ======= Fit mô hình ARIMA =======
model = ARIMA(ts_train, order=(2,1,2))  # order=(p,d,q) bạn có thể tinh chỉnh
model_fit = model.fit()

# ======= Dự đoán tiếp N bước trong tương lai =======
n_steps = len(test)
forecast = model_fit.forecast(steps=n_steps)

# ======= Làm mượt dự đoán bằng Kalman Filter =======
kf = KalmanFilter(initial_state=forecast.iloc[0])
smoothed = []

for z in forecast:
    kf.predict()
    smoothed_val = kf.update(z)
    smoothed.append(smoothed_val)

# ======= Ghi kết quả ra file submission =======
test['Predicted_Listening_Time_minutes'] = smoothed
test.to_csv("submission_ARIMA1.csv", index=False)